# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.123808e+02     1.509665e+00
 * time: 0.9006941318511963
     1     1.063659e+01     8.912954e-01
 * time: 2.385573148727417
     2    -1.204671e+01     9.046873e-01
 * time: 3.116708993911743
     3    -3.437072e+01     7.931602e-01
 * time: 4.205080986022949
     4    -4.816746e+01     6.137722e-01
 * time: 5.2208781242370605
     5    -5.699879e+01     2.243068e-01
 * time: 6.221042156219482
     6    -5.973594e+01     1.397788e-01
 * time: 6.88073205947876
     7    -6.078716e+01     4.847727e-02
 * time: 7.546424150466919
     8    -6.120001e+01     7.022580e-02
 * time: 8.221664190292358
     9    -6.150142e+01     4.655458e-02
 * time: 8.895681142807007
    10    -6.170934e+01     3.102695e-02
 * time: 9.572257995605469
    11    -6.188900e+01     2.151821e-02
 * time: 10.243652105331421
    12    -6.196809e+01     1.845573e-02
 * time: 10.90320110321045
    13    -6.205629e+01     1.428262e-02
 * time: 11.57168698310852
    1

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671051
    AtomicLocal         -18.8557612
    AtomicNonlocal      14.8522610
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485350 
    Xc                  -19.3336812

    total               -62.261666455555